In [8]:
import cv2
import math
import os
import numpy as np

def tile_video(
    video_path: str,
    tile_rows: int,
    tile_cols: int,
    start_frame: int,
    end_frame: int,
    frame_step: int = 1,
    output_dir: str = "./tiles",
    resize_width: int = None,
    resize_height: int = None
):
    """
    動画を読み込み、指定した開始フレームから終了フレームまでを
    frame_step間隔でフレームを抽出し、タイル状に配置した画像を出力する関数。

    Args:
        video_path (str): 入力動画のファイルパス
        tile_rows (int): タイルの縦方向の枚数
        tile_cols (int): タイルの横方向の枚数
        start_frame (int): 開始フレーム番号
        end_frame (int): 終了フレーム番号
        frame_step (int, optional): フレームを取得する間隔 (例: 2なら2フレームごとに1枚)
        output_dir (str, optional): タイル画像の出力先ディレクトリ。デフォルトは "./tiles"
        resize_width (int, optional): タイル化前に各フレームをリサイズする際の幅
        resize_height (int, optional): タイル化前に各フレームをリサイズする際の高さ

    Returns:
        None
        （タイル画像を指定ディレクトリにファイルとして出力します。）
    """
    
    # 出力先ディレクトリの作成
    os.makedirs(output_dir, exist_ok=True)

    # 動画キャプチャを開く
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"動画ファイルを開けませんでした: {video_path}")
        return

    # 動画の総フレーム数を取得
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # start_frame が範囲外の場合にクリップ
    start_frame = max(0, min(start_frame, total_frames - 1))
    # end_frame が範囲外の場合にクリップ
    end_frame = max(0, min(end_frame, total_frames - 1))

    # 開始フレームまで飛ばす
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frames = []
    current_frame = start_frame

    # フレームを読み込んでリストに格納 (frame_step間隔)
    while current_frame <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        
        # (current_frame - start_frame) で開始から何フレーム目かを確認
        # その値が frame_step の倍数ならフレームを取得する
        if (current_frame - start_frame) % frame_step == 0:
            # 必要に応じてリサイズ
            if resize_width and resize_height:
                frame = cv2.resize(frame, (resize_width, resize_height))
            frames.append(frame)
        
        current_frame += 1

    cap.release()

    # 取得したフレームがない場合は処理を終了
    if len(frames) == 0:
        print("指定された範囲およびステップに対応するフレームがありません。")
        return

    # タイル1枚あたりに配置できるフレーム数
    frames_per_tile = tile_rows * tile_cols

    # 何枚のタイル画像が必要か計算
    num_tiles = math.ceil(len(frames) / frames_per_tile)

    for tile_index in range(num_tiles):
        # タイル用に切り出すフレームを取得
        start_idx = tile_index * frames_per_tile
        end_idx = min((tile_index + 1) * frames_per_tile, len(frames))
        tile_frames = frames[start_idx:end_idx]

        # タイルの行列を作成
        # まず、タイル化する各フレームの画像サイズを取得
        tile_height, tile_width = tile_frames[0].shape[:2]

        # タイル全体のキャンバスを作成
        # (tile_rows*tile_height) × (tile_cols*tile_width) の大きさ
        tiled_image = np.zeros(
            (
                tile_rows * tile_height,
                tile_cols * tile_width,
                3
            ),
            dtype=np.uint8
        )

        # 各フレームをタイル状にコピー
        for i, f in enumerate(tile_frames):
            row_idx = i // tile_cols  # 何行目のタイルか
            col_idx = i % tile_cols   # 何列目のタイルか

            y_start = row_idx * tile_height
            y_end = y_start + tile_height
            x_start = col_idx * tile_width
            x_end = x_start + tile_width

            tiled_image[y_start:y_end, x_start:x_end, :] = f

        # タイル画像のファイル名を作成
        tile_filename = os.path.join(output_dir, f"tile_{tile_index+1:03d}.png")
        cv2.imwrite(tile_filename, tiled_image)
        print(f"タイル画像を出力しました: {tile_filename}")

    print("タイル作成が完了しました。")

In [9]:
# 例: 動画 "sample.mp4" からフレーム番号 0～499 の範囲を取得し、
# 2フレームごとに1枚を抽出。
# 抽出したフレームを (10行 × 5列) のタイル画像にして出力。
# フレームは幅160, 高さ90にリサイズして使用。

tile_video(
    video_path="sample.mp4",
    tile_rows=10,
    tile_cols=5,
    start_frame=0,
    end_frame=499,
    frame_step=2,           # 2フレームに1回
    output_dir="./output_tiles",
    resize_width=160,
    resize_height=90
)

タイル画像を出力しました: ./output_tiles\tile_001.png
タイル画像を出力しました: ./output_tiles\tile_002.png
タイル画像を出力しました: ./output_tiles\tile_003.png
タイル画像を出力しました: ./output_tiles\tile_004.png
タイル作成が完了しました。
